In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sn
import numpy as np
os.chdir('../')

In [ ]:
df = pd.read_csv('output/input.csv')

def produce_matrix(columns_list, names, filename, size=(7, 7)):
    confusion_matrix = []
    for i in columns_list:
        row = []
        for j in columns_list:
            equal = i.eq(j).sum()
            row.append(equal)

        confusion_matrix.append(row)


    plt.figure(figsize=size)
    mask = np.zeros_like(confusion_matrix)
    mask[np.triu_indices_from(mask, k=1)] = True

    with sn.axes_style("white"):

        sn.heatmap(confusion_matrix, mask=mask, annot=True, fmt="d", xticklabels=names, yticklabels=names)
        plt.yticks(rotation=0) 
        plt.savefig(f"output/{filename}.jpeg", bbox_inches='tight')
        plt.show()
        
        

## Number of emergency covid hospital admissions and those that can be identified in A&E attendances
### Any A&E attendance

In [ ]:
num_emergency_cov_admissions = df['emergency_covid_hospital_admission_date'].count()
ae_before = (df.loc[
    df["ae_before_date"].notnull()
, "patient_id"])
print(f'Number of emergency covid hospital admissions: {num_emergency_cov_admissions}')
print(f'Number with A&E attendance before admission: {len(ae_before)} ({round((len(ae_before)/num_emergency_cov_admissions)*100, 2)}%)')

ae = len(df[
    (df["ae_attendance_date"].notnull())
])

ae_with_hosp_discharge = len(df[
    (df["ae_attendance_hosp_discharge_date"].notnull())
])

summary_table = pd.DataFrame({
    "emergency covid hospital admission": num_emergency_cov_admissions,
    "ae attendance before admission": len(ae_before),
    "ae_attendance": ae, 
    "ae attendance with hospital discharge": ae_with_hosp_discharge,
    
}, index=["count"])

summary_table.to_csv("output/summary_table.csv")

print('---')
print(f'Number of A&E attendances: {ae}')

#### Method 1 - Discharge destination
Restrict to those that attended A&E and had a hospital discharge destination (hospital ward, emergency short stay ward, icu).

In [ ]:
ae_hosp_before = (df.loc[
    df["ae_before_with_hospital_discharge_date"].notnull()
, "patient_id"])

print(f'Number with A&E attendance with hospital discharge before admission: {len(ae_hosp_before)} ({round((len(ae_hosp_before)/num_emergency_cov_admissions)*100, 2)}%)')


# false posisitves
fp_ae_hosp_discharge = len(df[
    (df["ae_attendance_hosp_discharge_date"].notnull()) & 
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==0)

])

tp_ae_hosp_discharge = len(df[
    (df["ae_attendance_hosp_discharge_date"].notnull()) & 
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==1)

])

discharge_destination = [len(ae_hosp_before), round((len(ae_hosp_before)/num_emergency_cov_admissions)*100, 2),fp_ae_hosp_discharge, tp_ae_hosp_discharge, round((fp_ae_hosp_discharge/(fp_ae_hosp_discharge + tp_ae_hosp_discharge))*100,2)]



print(f'False positives (FDR): {fp_ae_hosp_discharge} ({round((fp_ae_hosp_discharge/(fp_ae_hosp_discharge + tp_ae_hosp_discharge))*100,2)}%)')

#### Method 2 - Diagnosis code
Restrict to those with a COVID-19 diagnosis code or respiratory-related diagnosis code from the [emergency care diagnosis refset](https://termbrowser.nhs.uk/?perspective=full&conceptId1=991411000000109&edition=uk-edition&release=v20220216&server=https://termbrowser.nhs.uk/sct-browser-api/snomed&langRefset=900000000000508004)

In [ ]:
ae_cov_before = (df.loc[
    df["ae_before_with_covid"]==1
, "patient_id"])

ae_resp_before = (df.loc[
    df["ae_before_with_resp"]==1
, "patient_id"])

print(f'Number with A&E attendance and covid diagnosis code before admission: {len(ae_cov_before)} ({round((len(ae_cov_before)/num_emergency_cov_admissions)*100, 2)}%)')
print(f'Number with A&E attendance and respiratory code before admission: {len(ae_resp_before)} ({round((len(ae_resp_before)/num_emergency_cov_admissions)*100, 2)}%)')

fp_ae_cov = len(df[
    (df["ae_cov"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae"]==0)
    
])

fp_ae_resp = len(df[
    (df["ae_resp"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae"]==0)
    
])

tp_ae_cov = len(df[
    (df["ae_cov"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae"]==1)
    
])

tp_ae_resp = len(df[
    (df["ae_resp"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae"]==1)
    
])

diagnosis_code_cov = [len(ae_cov_before), round((len(ae_cov_before)/num_emergency_cov_admissions)*100, 2), fp_ae_cov, tp_ae_cov, round((fp_ae_cov/(fp_ae_cov + tp_ae_cov))*100,2)]
diagnosis_code_resp = [len(ae_resp_before), round((len(ae_resp_before)/num_emergency_cov_admissions)*100, 2), fp_ae_resp, tp_ae_resp, round((len(ae_resp_before)/num_emergency_cov_admissions)*100, 2)]

print(f'False positives (FDR) - A&E attendance and covid diagnosis code: {fp_ae_cov} ({round((fp_ae_cov/(fp_ae_cov + tp_ae_cov))*100,2)}%)')
print(f'False positives (FDR) - A&E attendance and respiratory diagnosis code: {fp_ae_resp} ({round((fp_ae_resp/(fp_ae_resp + tp_ae_resp))*100,2)}%)')

#### Method 3 - Identification in primary care

Restrict to those with a probable or suspected COVID-19 code recorded in primary care in the 2 weeks prior to A&E attendance.

In [ ]:
ae_prim_care_probable_before = (df.loc[
    df["ae_before_with_prim_care_probable"]==1
, "patient_id"])

ae_prim_care_suspected_before = (df.loc[
    df["ae_before_with_prim_care_suspected"]==1
, "patient_id"])

print(f'Number with A&E attendance and recent suspected covid code in primary care before admission: {len(ae_prim_care_suspected_before)} ({round((len(ae_prim_care_suspected_before)/num_emergency_cov_admissions)*100, 2)}%)')
print(f'Number with A&E attendance and recent probable covid code in primary care before admission: {len(ae_prim_care_probable_before)} ({round((len(ae_prim_care_probable_before)/num_emergency_cov_admissions)*100, 2)}%)')

fp_ae_suspected = len(df[
    (df["suspected_covid_primary_care_before_ae"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae"]==0)    
])

fp_ae_probable = len(df[
    (df["covid_primary_care_before_ae"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae"]==0)
])

tp_ae_suspected = len(df[
    (df["suspected_covid_primary_care_before_ae"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae"]==1)    
])

tp_ae_probable = len(df[
    (df["covid_primary_care_before_ae"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae"]==1)
])

prim_care_suspected = [len(ae_prim_care_suspected_before), round((len(ae_prim_care_suspected_before)/num_emergency_cov_admissions)*100, 2), fp_ae_suspected, tp_ae_suspected, round((fp_ae_suspected/(fp_ae_suspected + tp_ae_suspected))*100,2)]
prim_care_probable = [len(ae_prim_care_probable_before), round((len(ae_prim_care_probable_before)/num_emergency_cov_admissions)*100, 2), fp_ae_probable, tp_ae_probable, round((fp_ae_probable/(fp_ae_probable + tp_ae_probable))*100,2)]

print(f'False positives (FDR) - A&E attendance and probable covid code in primary care: {fp_ae_probable} ({round((fp_ae_probable/(fp_ae_probable + tp_ae_probable))*100,2)}%)')
print(f'False positives (FDR) - A&E attendance and suspected covid code in primary care: {fp_ae_suspected} ({round((fp_ae_suspected/(fp_ae_suspected + tp_ae_suspected))*100,2)}%)')

#### Method 4 - Positive test result

Restrict to those with a positive COVID-19 test result recorded up to 4 weeks prior to A&E attendance.

In [ ]:
ae_pos_test_before_2_weeks = (df.loc[
    df["ae_before_with_positive_covid_test_2_weeks"]==1
, "patient_id"])

ae_pos_test_before = (df.loc[
    df["ae_before_with_positive_covid_test"]==1
, "patient_id"])


print(f'Number with A&E attendance and recent positive test (2 weeks) before admission: {len(ae_pos_test_before_2_weeks)} ({round((len(ae_pos_test_before_2_weeks)/num_emergency_cov_admissions)*100, 2)}%)')
print(f'Number with A&E attendance and recent positive test (4 weeks) before admissio: {len(ae_pos_test_before)} ({round((len(ae_pos_test_before)/num_emergency_cov_admissions)*100, 2)}%)')

fp_ae_test = len(df[
    (df["pos_test"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae"]==0)
    
])

fp_ae_test_2_weeks = len(df[
    (df["pos_test_2_weeks"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae"]==0)
    
])

tp_ae_test = len(df[
    (df["pos_test"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae"]==1)
    
])

tp_ae_test_2_weeks = len(df[
    (df["pos_test_2_weeks"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae"]==1)
    
])
print(f'False positives (FDR) - A&E attendance and recent positive test (2 weeks): {fp_ae_test_2_weeks} ({round((fp_ae_test_2_weeks/(fp_ae_test_2_weeks + tp_ae_test_2_weeks))*100,2)}%)')
print(f'False positives (FDR) - A&E attendance and recent positive test (4 weeks): {fp_ae_test} ({round((fp_ae_test/(fp_ae_test + tp_ae_test))*100,2)}%)')

positive_test_2_weeks = [len(ae_pos_test_before_2_weeks), round((len(ae_pos_test_before_2_weeks)/num_emergency_cov_admissions)*100, 2), fp_ae_test_2_weeks, tp_ae_test_2_weeks, round((fp_ae_test_2_weeks/(fp_ae_test_2_weeks + tp_ae_test_2_weeks))*100,2)]
positive_test_4_weeks = [len(ae_pos_test_before), round((len(ae_pos_test_before)/num_emergency_cov_admissions)*100, 2), fp_ae_test, tp_ae_test, round((fp_ae_test/(fp_ae_test + tp_ae_test))*100,2)]


####  Combining approaches

##### Diagnosis codes (resp or cov) with recent positive test

In [ ]:
ae_hos_resp_cov_plus_test_2_weeks = (df.loc[(
        (df["ae_before_with_covid"]==1) | 
        (df["ae_before_with_resp"]==1) 
    )&
    (df["ae_before_with_positive_covid_test_2_weeks"]==1)
, "patient_id"])

fp_ae_hos_resp_cov_plus_test_2_weeks = len(df[
    (
        (df["ae_cov"]==1) |
        (df["ae_resp"]==1)    
    )&
    (df["pos_test_2_weeks"]==1)&
    (df["emergency_covid_hospital_admission_after_ae"]==0)
    
])

tp_ae_hos_resp_cov_plus_test_2_weeks = len(df[
    (
        (df["ae_cov"]==1) |
        (df["ae_resp"]==1)    
    )&
    (df["pos_test_2_weeks"]==1)&
    (df["emergency_covid_hospital_admission_after_ae"]==1)
    
])

print(f'Number with A&E attendance with either covid or respiratory diagnosis code and recent positive test (2 weeks) before admission: {len(ae_hos_resp_cov_plus_test_2_weeks)} ({round((len(ae_hos_resp_cov_plus_test_2_weeks)/num_emergency_cov_admissions)*100, 2)}%)')
print(f'False positives (FDR): {fp_ae_hos_resp_cov_plus_test_2_weeks} ({round((fp_ae_hos_resp_cov_plus_test_2_weeks/(fp_ae_hos_resp_cov_plus_test_2_weeks + tp_ae_hos_resp_cov_plus_test_2_weeks))*100,2)}%)')

code_plus_test = [len(ae_hos_resp_cov_plus_test_2_weeks), round((len(ae_hos_resp_cov_plus_test_2_weeks)/num_emergency_cov_admissions)*100, 2), fp_ae_hos_resp_cov_plus_test_2_weeks, tp_ae_hos_resp_cov_plus_test_2_weeks, round((fp_ae_hos_resp_cov_plus_test_2_weeks/(fp_ae_hos_resp_cov_plus_test_2_weeks + tp_ae_hos_resp_cov_plus_test_2_weeks))*100,2)]

##### Covid in primary care with recent positive test

In [ ]:
ae_hosp_prim_care_before_plus_2_week_test = (df.loc[
    
    
    (
        (df["ae_before_with_prim_care_suspected"]==1) | 
        (df["ae_before_with_prim_care_probable"]==1) 
    )
&
    (df["ae_before_with_positive_covid_test_2_weeks"]==1)
, "patient_id"])

fp_ae_hosp_prim_care_before_plus_2_week_test = len(df[
    (
        (df["covid_primary_care_before_ae"]==1) |
        (df["suspected_covid_primary_care_before_ae"]==1)    
    )&
    (df["pos_test_2_weeks"]==1)&
    (df["emergency_covid_hospital_admission_after_ae"]==0)
    
])

tp_ae_hosp_prim_care_before_plus_2_week_test = len(df[
    (
        (df["covid_primary_care_before_ae"]==1) |
        (df["suspected_covid_primary_care_before_ae"]==1)    
    )&
    (df["pos_test_2_weeks"]==1)&
    (df["emergency_covid_hospital_admission_after_ae"]==1)
    
])

print(f'Number with A&E attendance with suspected or probable covid in primary care and recent positive test (2 weeks) before admission: {len(ae_hosp_prim_care_before_plus_2_week_test)} ({round((len(ae_hosp_prim_care_before_plus_2_week_test)/num_emergency_cov_admissions)*100, 2)}%)')
print(f'False positives (FDR): {fp_ae_hosp_prim_care_before_plus_2_week_test} ({round((fp_ae_hosp_prim_care_before_plus_2_week_test/(fp_ae_hosp_prim_care_before_plus_2_week_test + tp_ae_hosp_prim_care_before_plus_2_week_test))*100,2)}%)')

primary_care_plus_test = [len(ae_hosp_prim_care_before_plus_2_week_test), round((len(ae_hosp_prim_care_before_plus_2_week_test)/num_emergency_cov_admissions)*100, 2), fp_ae_hosp_prim_care_before_plus_2_week_test, tp_ae_hosp_prim_care_before_plus_2_week_test, round((fp_ae_hosp_prim_care_before_plus_2_week_test/(fp_ae_hosp_prim_care_before_plus_2_week_test + tp_ae_hosp_prim_care_before_plus_2_week_test))*100,2)]

##### Recent positive test and either covid in primary care or diagnosis code (resp or cov)

In [ ]:
combined_before = (df.loc[
        (df["ae_before_with_positive_covid_test_2_weeks"]==1) &
        (
            (df["ae_before_with_prim_care_suspected"]==1) | 
            (df["ae_before_with_prim_care_probable"]==1) |
            (df["ae_before_with_covid"]==1) | 
            (df["ae_before_with_resp"]==1)   
        )
, "patient_id"])
fp_combined_before = len(df[
    (
        (df["covid_primary_care_before_ae"]==1) |
        (df["suspected_covid_primary_care_before_ae"]==1) |
        (df["ae_cov"]==1) |
        (df["ae_resp"]==1)
        
        
    )&
    (df["pos_test_2_weeks"]==1)&
    (df["emergency_covid_hospital_admission_after_ae"]==0)
    
])

tp_combined_before = len(df[
    (
        (df["covid_primary_care_before_ae"]==1) |
        (df["suspected_covid_primary_care_before_ae"]==1) |
        (df["ae_cov"]==1) |
        (df["ae_resp"]==1)
    )&
    (df["pos_test_2_weeks"]==1)&
    (df["emergency_covid_hospital_admission_after_ae"]==1)
    
])


print(f'Number with A&E attendance with recent positive test and either suspected or probable covid in primary care or a covid or respiratory diagnosis code on attendance: {len(combined_before)} ({round((len(combined_before)/num_emergency_cov_admissions)*100, 2)}%)')
print(f'False positives (FDR): {fp_combined_before} ({round((fp_combined_before/(fp_combined_before + tp_combined_before))*100,2)}%)')

pos_test_prim_care_or_diagnosis_code = [len(combined_before), round((len(combined_before)/num_emergency_cov_admissions)*100, 2), fp_combined_before, tp_combined_before, round((fp_combined_before/(fp_combined_before + tp_combined_before))*100,2)]

In [ ]:
identification_table_index = ["count", "sens", "fp", "tp", "fdr"]
identification_table = pd.DataFrame({
    "discharge_destination": pd.Series(discharge_destination,index=identification_table_index),
    "covid_diagnosis_code": pd.Series(diagnosis_code_cov,index=identification_table_index),
    "resp_diagnosis_code": pd.Series(diagnosis_code_resp,index=identification_table_index),
    "primary_care_suspected": pd.Series(prim_care_suspected,index=identification_table_index),
    "primary_care_probable": pd.Series(prim_care_probable,index=identification_table_index),
    "positive_test_2_weeks": pd.Series(positive_test_2_weeks,index=identification_table_index),
    "positive_test_4_weeks": pd.Series(positive_test_4_weeks,index=identification_table_index),
    "diagnosis_code_with_positive_test": pd.Series(code_plus_test,index=identification_table_index),
    "primary_care_with_positive_test": pd.Series(primary_care_plus_test,index=identification_table_index),
    "pos_test_prim_care_or_diagnosis": pd.Series(pos_test_prim_care_or_diagnosis_code,index=identification_table_index)
    
})
identification_table.to_csv("output/identification_table_no_hosp_discharge.csv")

In [ ]:
df = pd.read_csv('output/input.csv')
        
produce_matrix(columns_list = [ae_hosp_before, ae_cov_before, ae_resp_before, ae_prim_care_probable_before, ae_prim_care_suspected_before, ae_pos_test_before_2_weeks, ae_pos_test_before, ae_hos_resp_cov_plus_test_2_weeks, ae_hosp_prim_care_before_plus_2_week_test], names=["Hospital discharge", "Covid diagnosis code", "Respiratory diagnosis code", "Probable covid code", "Suspected covid code", "Recent positive test (2 weeks)", "Recent positive test (4 weeks)", "Diagnosis codes (resp or cov) with recent positive test", "Covid in primary care with recent positive test", "Recent positive test and either covid in primary care or diagnosis code (resp or cov)"], filename="cm_no_hosp_discharge")


### Restrict to A&E attendance with hospital discharge

In [ ]:
ae_with_hosp_discharge = len(df[
    (df["ae_attendance_hosp_discharge_date"].notnull())
])
print(f'Number of A&E attendances with hospital discharge: {ae_with_hosp_discharge}')

#### Method 1 - Discharge destination
Restrict to those that attended A&E and had a hospital discharge destination (hospital ward, emergency short stay ward, icu).

In [ ]:
ae_hosp_before = (df.loc[
    df["ae_before_with_hospital_discharge_date"].notnull()
, "patient_id"])

print(f'Number with A&E attendance with hospital discharge before admission: {len(ae_hosp_before)} ({round((len(ae_hosp_before)/num_emergency_cov_admissions)*100, 2)}%)')


# false posisitves
fp_ae_hosp_discharge = len(df[
    (df["ae_attendance_hosp_discharge_date"].notnull()) & 
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==0)

])

tp_ae_hosp_discharge = len(df[
    (df["ae_attendance_hosp_discharge_date"].notnull()) & 
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==1)

])
discharge_destination = [len(ae_hosp_before), round((len(ae_hosp_before)/num_emergency_cov_admissions)*100, 2),fp_ae_hosp_discharge, tp_ae_hosp_discharge, round((fp_ae_hosp_discharge/(fp_ae_hosp_discharge + tp_ae_hosp_discharge))*100,2)]

print(f'False positives (FDR): {fp_ae_hosp_discharge} ({round((fp_ae_hosp_discharge/(fp_ae_hosp_discharge + tp_ae_hosp_discharge))*100,2)}%)')

#### Method 2 - Diagnosis code
Restrict to those with a COVID-19 diagnosis code or respiratory-related diagnosis code from the [emergency care diagnosis refset](https://termbrowser.nhs.uk/?perspective=full&conceptId1=991411000000109&edition=uk-edition&release=v20220216&server=https://termbrowser.nhs.uk/sct-browser-api/snomed&langRefset=900000000000508004)

In [ ]:
ae_cov_before = (df.loc[
    df["ae_before_with_hospital_discharge_covid"]==1
, "patient_id"])

ae_resp_before = (df.loc[
    df["ae_before_with_hospital_discharge_resp"]==1
, "patient_id"])

print(f'Number with A&E attendance and covid diagnosis code before admission: {len(ae_cov_before)} ({round((len(ae_cov_before)/num_emergency_cov_admissions)*100, 2)}%)')
print(f'Number with A&E attendance and respiratory code before admission: {len(ae_resp_before)} ({round((len(ae_resp_before)/num_emergency_cov_admissions)*100, 2)}%)')

fp_ae_cov = len(df[
    (df["ae_cov_hosp_discharge"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==0)
    
])

fp_ae_resp = len(df[
    (df["ae_resp_hosp_discharge"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==0)
    
])

tp_ae_cov = len(df[
    (df["ae_cov_hosp_discharge"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==1)
    
])

tp_ae_resp = len(df[
    (df["ae_resp_hosp_discharge"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==1)
    
])

diagnosis_code_cov = [len(ae_cov_before), round((len(ae_cov_before)/num_emergency_cov_admissions)*100, 2), fp_ae_cov, tp_ae_cov, round((fp_ae_cov/(fp_ae_cov + tp_ae_cov))*100,2)]
diagnosis_code_resp = [len(ae_resp_before), round((len(ae_resp_before)/num_emergency_cov_admissions)*100, 2), fp_ae_resp, tp_ae_resp, round((len(ae_resp_before)/num_emergency_cov_admissions)*100, 2)]


print(f'False positives (FDR) - A&E attendance and covid diagnosis code: {fp_ae_cov} ({round((fp_ae_cov/(fp_ae_cov + tp_ae_cov))*100,2)}%)')
print(f'False positives (FDR) - A&E attendance and respiratory diagnosis code: {fp_ae_resp} ({round((fp_ae_resp/(fp_ae_resp + tp_ae_resp))*100,2)}%)')

#### Method 3 - Identification in primary care

Restrict to those with a probable or suspected COVID-19 code recorded in primary care in the 2 weeks prior to A&E attendance.

In [ ]:
ae_prim_care_probable_before = (df.loc[
    df["ae_before_with_hospital_discharge_prim_care_probable"]==1
, "patient_id"])

ae_prim_care_suspected_before = (df.loc[
    df["ae_before_with_hospital_discharge_prim_care_suspected"]==1
, "patient_id"])

print(f'Number with A&E attendance and recent suspected covid code in primary care before admission: {len(ae_prim_care_suspected_before)} ({round((len(ae_prim_care_suspected_before)/num_emergency_cov_admissions)*100, 2)}%)')
print(f'Number with A&E attendance and recent probable covid code in primary care before admission: {len(ae_prim_care_probable_before)} ({round((len(ae_prim_care_probable_before)/num_emergency_cov_admissions)*100, 2)}%)')

fp_ae_suspected = len(df[
    (df["suspected_covid_primary_care_before_ae_hosp_discharge"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==0)    
])

fp_ae_probable = len(df[
    (df["covid_primary_care_before_ae_hosp_discharge"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==0)
])

tp_ae_suspected = len(df[
    (df["suspected_covid_primary_care_before_ae_hosp_discharge"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==1)    
])

tp_ae_probable = len(df[
    (df["covid_primary_care_before_ae_hosp_discharge"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==1)
])

prim_care_suspected = [len(ae_prim_care_suspected_before), round((len(ae_prim_care_suspected_before)/num_emergency_cov_admissions)*100, 2), fp_ae_suspected, tp_ae_suspected, round((fp_ae_suspected/(fp_ae_suspected + tp_ae_suspected))*100,2)]
prim_care_probable = [len(ae_prim_care_probable_before), round((len(ae_prim_care_probable_before)/num_emergency_cov_admissions)*100, 2), fp_ae_probable, tp_ae_probable, round((fp_ae_probable/(fp_ae_probable + tp_ae_probable))*100,2)]


print(f'False positives (FDR) - A&E attendance and probable covid code in primary care: {fp_ae_probable} ({round((fp_ae_probable/(fp_ae_probable + tp_ae_probable))*100,2)}%)')
print(f'False positives (FDR) - A&E attendance and suspected covid code in primary care: {fp_ae_suspected} ({round((fp_ae_suspected/(fp_ae_suspected + tp_ae_suspected))*100,2)}%)')

#### Method 4 - Positive test result

Restrict to those with a positive COVID-19 test result recorded up to 4 weeks prior to A&E attendance.

In [ ]:
ae_pos_test_before_2_weeks = (df.loc[
    df["ae_before_with_hospital_discharge_positive_covid_test_2_weeks"]==1
, "patient_id"])

ae_pos_test_before = (df.loc[
    df["ae_before_with_hospital_discharge_positive_covid_test"]==1
, "patient_id"])


print(f'Number with A&E attendance and recent positive test (2 weeks) before admission: {len(ae_pos_test_before_2_weeks)} ({round((len(ae_pos_test_before_2_weeks)/num_emergency_cov_admissions)*100, 2)}%)')
print(f'Number with A&E attendance and recent positive test (4 weeks) before admissio: {len(ae_pos_test_before)} ({round((len(ae_pos_test_before)/num_emergency_cov_admissions)*100, 2)}%)')

fp_ae_test = len(df[
    (df["pos_test_hosp_discharge"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==0)
    
])

fp_ae_test_2_weeks = len(df[
    (df["pos_test_2_weeks_hosp_discharge"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==0)
    
])

tp_ae_test = len(df[
    (df["pos_test_hosp_discharge"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==1)
    
])

tp_ae_test_2_weeks = len(df[
    (df["pos_test_2_weeks_hosp_discharge"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==1)
    
])

positive_test_2_weeks = [len(ae_pos_test_before_2_weeks), round((len(ae_pos_test_before_2_weeks)/num_emergency_cov_admissions)*100, 2), fp_ae_test_2_weeks, tp_ae_test_2_weeks, round((fp_ae_test_2_weeks/(fp_ae_test_2_weeks + tp_ae_test_2_weeks))*100,2)]
positive_test_4_weeks = [len(ae_pos_test_before), round((len(ae_pos_test_before)/num_emergency_cov_admissions)*100, 2), fp_ae_test, tp_ae_test, round((fp_ae_test/(fp_ae_test + tp_ae_test))*100,2)]


print(f'False positives (FDR) - A&E attendance and recent positive test (2 weeks): {fp_ae_test_2_weeks} ({round((fp_ae_test_2_weeks/(fp_ae_test_2_weeks + tp_ae_test_2_weeks))*100,2)}%)')
print(f'False positives (FDR) - A&E attendance and recent positive test (4 weeks): {fp_ae_test} ({round((fp_ae_test/(fp_ae_test + tp_ae_test))*100,2)}%)')

####  Combining approaches

##### Diagnosis codes (resp or cov) with recent positive test

In [ ]:
ae_hos_resp_cov_plus_test_2_weeks = (df.loc[(
        (df["ae_before_with_hospital_discharge_covid"]==1) | 
        (df["ae_before_with_hospital_discharge_resp"]==1) 
    )&
    (df["ae_before_with_hospital_discharge_positive_covid_test_2_weeks"]==1)
, "patient_id"])

fp_ae_hos_resp_cov_plus_test_2_weeks = len(df[
    (
        (df["ae_cov_hosp_discharge"]==1) |
        (df["ae_resp_hosp_discharge"]==1)    
    )&
    (df["pos_test_2_weeks_hosp_discharge"]==1)&
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==0)
    
])

tp_ae_hos_resp_cov_plus_test_2_weeks = len(df[
    (
        (df["ae_cov_hosp_discharge"]==1) |
        (df["ae_resp_hosp_discharge"]==1)    
    )&
    (df["pos_test_2_weeks_hosp_discharge"]==1)&
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==1)
    
])

print(f'Number with A&E attendance with either covid or respiratory diagnosis code and recent positive test (2 weeks) before admission: {len(ae_hos_resp_cov_plus_test_2_weeks)} ({round((len(ae_hos_resp_cov_plus_test_2_weeks)/num_emergency_cov_admissions)*100, 2)}%)')
print(f'False positives (FDR): {fp_ae_hos_resp_cov_plus_test_2_weeks} ({round((fp_ae_hos_resp_cov_plus_test_2_weeks/(fp_ae_hos_resp_cov_plus_test_2_weeks + tp_ae_hos_resp_cov_plus_test_2_weeks))*100,2)}%)')
code_plus_test = [len(ae_hos_resp_cov_plus_test_2_weeks), round((len(ae_hos_resp_cov_plus_test_2_weeks)/num_emergency_cov_admissions)*100, 2), fp_ae_hos_resp_cov_plus_test_2_weeks, tp_ae_hos_resp_cov_plus_test_2_weeks, round((fp_ae_hos_resp_cov_plus_test_2_weeks/(fp_ae_hos_resp_cov_plus_test_2_weeks + tp_ae_hos_resp_cov_plus_test_2_weeks))*100,2)]

##### Covid in primary care with recent positive test

In [ ]:
ae_hosp_prim_care_before_plus_2_week_test = (df.loc[(
        (df["ae_before_with_hospital_discharge_prim_care_suspected"]==1) | 
        (df["ae_before_with_hospital_discharge_prim_care_probable"]==1) 
    )
&
    (df["ae_before_with_hospital_discharge_positive_covid_test_2_weeks"]==1)
, "patient_id"])

fp_ae_hosp_prim_care_before_plus_2_week_test = len(df[
    (
        (df["covid_primary_care_before_ae_hosp_discharge"]==1) |
        (df["suspected_covid_primary_care_before_ae_hosp_discharge"]==1)    
    )&
    (df["pos_test_2_weeks_hosp_discharge"]==1)&
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==0)
    
])

tp_ae_hosp_prim_care_before_plus_2_week_test = len(df[
    (
        (df["covid_primary_care_before_ae_hosp_discharge"]==1) |
        (df["suspected_covid_primary_care_before_ae_hosp_discharge"]==1)    
    )&
    (df["pos_test_2_weeks_hosp_discharge"]==1)&
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==1)
    
])

print(f'Number with A&E attendance with suspected or probable covid in primary care and recent positive test (2 weeks) before admission: {len(ae_hosp_prim_care_before_plus_2_week_test)} ({round((len(ae_hosp_prim_care_before_plus_2_week_test)/num_emergency_cov_admissions)*100, 2)}%)')
print(f'False positives (FDR): {fp_ae_hosp_prim_care_before_plus_2_week_test} ({round((fp_ae_hosp_prim_care_before_plus_2_week_test/(fp_ae_hosp_prim_care_before_plus_2_week_test + tp_ae_hosp_prim_care_before_plus_2_week_test))*100,2)}%)')

primary_care_plus_test = [len(ae_hosp_prim_care_before_plus_2_week_test), round((len(ae_hosp_prim_care_before_plus_2_week_test)/num_emergency_cov_admissions)*100, 2), fp_ae_hosp_prim_care_before_plus_2_week_test, tp_ae_hosp_prim_care_before_plus_2_week_test, round((fp_ae_hosp_prim_care_before_plus_2_week_test/(fp_ae_hosp_prim_care_before_plus_2_week_test + tp_ae_hosp_prim_care_before_plus_2_week_test))*100,2)]

##### Recent positive test and either covid in primary care or diagnosis code (resp or cov)

In [ ]:
combined_before = (df.loc[
        (df["ae_before_with_hospital_discharge_positive_covid_test_2_weeks"]==1) &
        (
            (df["ae_before_with_hospital_discharge_prim_care_suspected"]==1) | 
            (df["ae_before_with_hospital_discharge_prim_care_probable"]==1) |
            (df["ae_before_with_hospital_discharge_covid"]==1) | 
            (df["ae_before_with_hospital_discharge_resp"]==1)   
        )
, "patient_id"])
fp_combined_before = len(df[
    (
        (df["covid_primary_care_before_ae_hosp_discharge"]==1) |
        (df["suspected_covid_primary_care_before_ae_hosp_discharge"]==1) |
        (df["ae_cov_hosp_discharge"]==1) |
        (df["ae_resp_hosp_discharge"]==1)
        
        
    )&
    (df["pos_test_2_weeks_hosp_discharge"]==1)&
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==0)
    
])

tp_combined_before = len(df[
    (
        (df["covid_primary_care_before_ae_hosp_discharge"]==1) |
        (df["suspected_covid_primary_care_before_ae_hosp_discharge"]==1) |
        (df["ae_cov_hosp_discharge"]==1) |
        (df["ae_resp_hosp_discharge"]==1)
    )&
    (df["pos_test_2_weeks_hosp_discharge"]==1)&
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==1)
    
])


print(f'Number with A&E attendance with recent positive test and either suspected or probable covid in primary care or a covid or respiratory diagnosis code on attendance: {len(combined_before)} ({round((len(combined_before)/num_emergency_cov_admissions)*100, 2)}%)')
print(f'False positives (FDR): {fp_combined_before} ({round((fp_combined_before/(fp_combined_before + tp_combined_before))*100,2)}%)')
pos_test_prim_care_or_diagnosis_code = [len(combined_before), round((len(combined_before)/num_emergency_cov_admissions)*100, 2), fp_combined_before, tp_combined_before, round((fp_combined_before/(fp_combined_before + tp_combined_before))*100,2)]

In [ ]:
identification_table_index = ["count", "sens", "fp", "tp", "fdr"]
identification_table = pd.DataFrame({
    "discharge_destination": pd.Series(discharge_destination,index=identification_table_index),
    "covid_diagnosis_code": pd.Series(diagnosis_code_cov,index=identification_table_index),
    "resp_diagnosis_code": pd.Series(diagnosis_code_resp,index=identification_table_index),
    "primary_care_suspected": pd.Series(prim_care_suspected,index=identification_table_index),
    "primary_care_probable": pd.Series(prim_care_probable,index=identification_table_index),
    "positive_test_2_weeks": pd.Series(positive_test_2_weeks,index=identification_table_index),
    "positive_test_4_weeks": pd.Series(positive_test_4_weeks,index=identification_table_index),
    "diagnosis_code_with_positive_test": pd.Series(code_plus_test,index=identification_table_index),
    "primary_care_with_positive_test": pd.Series(primary_care_plus_test,index=identification_table_index),
    "pos_test_prim_care_or_diagnosis": pd.Series(pos_test_prim_care_or_diagnosis_code,index=identification_table_index)
    
})
identification_table.to_csv("output/identification_table_hosp_discharge.csv")

In [ ]:
produce_matrix(columns_list = [ae_hosp_before, ae_cov_before, ae_resp_before, ae_prim_care_probable_before, ae_prim_care_suspected_before, ae_pos_test_before_2_weeks, ae_pos_test_before, ae_hos_resp_cov_plus_test_2_weeks, ae_hosp_prim_care_before_plus_2_week_test], names=["Hospital discharge", "Covid diagnosis code", "Respiratory diagnosis code", "Probable covid code", "Suspected covid code", "Recent positive test (2 weeks)", "Recent positive test (4 weeks)", "Diagnosis codes (resp or cov) with recent positive test", "Covid in primary care with recent positive test", "Recent positive test and either covid in primary care or diagnosis code (resp or cov)"], filename="cm_hosp_discharge")

## Number of emergency primary covid hospital admissions and those that can be identified in A&E attendances
### Any A&E attendance

In [ ]:
df = pd.read_csv('output/input_prim_cov.csv')


num_emergency_cov_admissions = df['emergency_covid_hospital_admission_date'].count()
ae_before = (df.loc[
    df["ae_before_date"].notnull()
, "patient_id"])
print(f'Number of emergency primary covid hospital admissions: {num_emergency_cov_admissions}')
print(f'Number with A&E attendance before admission: {len(ae_before)} ({round((len(ae_before)/num_emergency_cov_admissions)*100, 2)}%)')

ae = len(df[
    (df["ae_attendance_date"].notnull())
])

ae_with_hosp_discharge = len(df[
    (df["ae_attendance_hosp_discharge_date"].notnull())
])

summary_table = pd.DataFrame({
    "emergency covid hospital admission": num_emergency_cov_admissions,
    "ae attendance before admission": len(ae_before),
    "ae_attendance": ae, 
    "ae attendance with hospital discharge": ae_with_hosp_discharge,
    
}, index=["count"])

summary_table.to_csv("output/summary_table_primary.csv")

print('---')
print(f'Number of A&E attendances: {ae}')

#### Method 1 - Discharge destination
Restrict to those that attended A&E and had a hospital discharge destination (hospital ward, emergency short stay ward, icu).

In [ ]:
ae_hosp_before = (df.loc[
    df["ae_before_with_hospital_discharge_date"].notnull()
, "patient_id"])

print(f'Number with A&E attendance with hospital discharge before admission: {len(ae_hosp_before)} ({round((len(ae_hosp_before)/num_emergency_cov_admissions)*100, 2)}%)')


# false posisitves
fp_ae_hosp_discharge = len(df[
    (df["ae_attendance_hosp_discharge_date"].notnull()) & 
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==0)

])

tp_ae_hosp_discharge = len(df[
    (df["ae_attendance_hosp_discharge_date"].notnull()) & 
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==1)

])

discharge_destination = [len(ae_hosp_before), round((len(ae_hosp_before)/num_emergency_cov_admissions)*100, 2),fp_ae_hosp_discharge, tp_ae_hosp_discharge, round((fp_ae_hosp_discharge/(fp_ae_hosp_discharge + tp_ae_hosp_discharge))*100,2)]

print(f'False positives (FDR): {fp_ae_hosp_discharge} ({round((fp_ae_hosp_discharge/(fp_ae_hosp_discharge + tp_ae_hosp_discharge))*100,2)}%)')

#### Method 2 - Diagnosis code
Restrict to those with a COVID-19 diagnosis code or respiratory-related diagnosis code from the [emergency care diagnosis refset](https://termbrowser.nhs.uk/?perspective=full&conceptId1=991411000000109&edition=uk-edition&release=v20220216&server=https://termbrowser.nhs.uk/sct-browser-api/snomed&langRefset=900000000000508004)

In [ ]:
ae_cov_before = (df.loc[
    df["ae_before_with_covid"]==1
, "patient_id"])

ae_resp_before = (df.loc[
    df["ae_before_with_resp"]==1
, "patient_id"])

print(f'Number with A&E attendance and covid diagnosis code before admission: {len(ae_cov_before)} ({round((len(ae_cov_before)/num_emergency_cov_admissions)*100, 2)}%)')
print(f'Number with A&E attendance and respiratory code before admission: {len(ae_resp_before)} ({round((len(ae_resp_before)/num_emergency_cov_admissions)*100, 2)}%)')

fp_ae_cov = len(df[
    (df["ae_cov"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae"]==0)
    
])

fp_ae_resp = len(df[
    (df["ae_resp"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae"]==0)
    
])

tp_ae_cov = len(df[
    (df["ae_cov"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae"]==1)
    
])

tp_ae_resp = len(df[
    (df["ae_resp"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae"]==1)
    
])


diagnosis_code_cov = [len(ae_cov_before), round((len(ae_cov_before)/num_emergency_cov_admissions)*100, 2), fp_ae_cov, tp_ae_cov, round((fp_ae_cov/(fp_ae_cov + tp_ae_cov))*100,2)]
diagnosis_code_resp = [len(ae_resp_before), round((len(ae_resp_before)/num_emergency_cov_admissions)*100, 2), fp_ae_resp, tp_ae_resp, round((len(ae_resp_before)/num_emergency_cov_admissions)*100, 2)]


print(f'False positives (FDR) - A&E attendance and covid diagnosis code: {fp_ae_cov} ({round((fp_ae_cov/(fp_ae_cov + tp_ae_cov))*100,2)}%)')
print(f'False positives (FDR) - A&E attendance and respiratory diagnosis code: {fp_ae_resp} ({round((fp_ae_resp/(fp_ae_resp + tp_ae_resp))*100,2)}%)')

#### Method 3 - Identification in primary care

Restrict to those with a probable or suspected COVID-19 code recorded in primary care in the 2 weeks prior to A&E attendance.

In [ ]:
ae_prim_care_probable_before = (df.loc[
    df["ae_before_with_prim_care_probable"]==1
, "patient_id"])

ae_prim_care_suspected_before = (df.loc[
    df["ae_before_with_prim_care_suspected"]==1
, "patient_id"])

print(f'Number with A&E attendance and recent suspected covid code in primary care before admission: {len(ae_prim_care_suspected_before)} ({round((len(ae_prim_care_suspected_before)/num_emergency_cov_admissions)*100, 2)}%)')
print(f'Number with A&E attendance and recent probable covid code in primary care before admission: {len(ae_prim_care_probable_before)} ({round((len(ae_prim_care_probable_before)/num_emergency_cov_admissions)*100, 2)}%)')

fp_ae_suspected = len(df[
    (df["suspected_covid_primary_care_before_ae"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae"]==0)    
])

fp_ae_probable = len(df[
    (df["covid_primary_care_before_ae"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae"]==0)
])

tp_ae_suspected = len(df[
    (df["suspected_covid_primary_care_before_ae"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae"]==1)    
])

tp_ae_probable = len(df[
    (df["covid_primary_care_before_ae"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae"]==1)
])

prim_care_suspected = [len(ae_prim_care_suspected_before), round((len(ae_prim_care_suspected_before)/num_emergency_cov_admissions)*100, 2), fp_ae_suspected, tp_ae_suspected, round((fp_ae_suspected/(fp_ae_suspected + tp_ae_suspected))*100,2)]
prim_care_probable = [len(ae_prim_care_probable_before), round((len(ae_prim_care_probable_before)/num_emergency_cov_admissions)*100, 2), fp_ae_probable, tp_ae_probable, round((fp_ae_probable/(fp_ae_probable + tp_ae_probable))*100,2)]


print(f'False positives (FDR) - A&E attendance and probable covid code in primary care: {fp_ae_probable} ({round((fp_ae_probable/(fp_ae_probable + tp_ae_probable))*100,2)}%)')
print(f'False positives (FDR) - A&E attendance and suspected covid code in primary care: {fp_ae_suspected} ({round((fp_ae_suspected/(fp_ae_suspected + tp_ae_suspected))*100,2)}%)')

#### Method 4 - Positive test result

Restrict to those with a positive COVID-19 test result recorded up to 4 weeks prior to A&E attendance.

In [ ]:
ae_pos_test_before_2_weeks = (df.loc[
    df["ae_before_with_positive_covid_test_2_weeks"]==1
, "patient_id"])

ae_pos_test_before = (df.loc[
    df["ae_before_with_positive_covid_test"]==1
, "patient_id"])


print(f'Number with A&E attendance and recent positive test (2 weeks) before admission: {len(ae_pos_test_before_2_weeks)} ({round((len(ae_pos_test_before_2_weeks)/num_emergency_cov_admissions)*100, 2)}%)')
print(f'Number with A&E attendance and recent positive test (4 weeks) before admissio: {len(ae_pos_test_before)} ({round((len(ae_pos_test_before)/num_emergency_cov_admissions)*100, 2)}%)')

fp_ae_test = len(df[
    (df["pos_test"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae"]==0)
    
])

fp_ae_test_2_weeks = len(df[
    (df["pos_test_2_weeks"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae"]==0)
    
])

tp_ae_test = len(df[
    (df["pos_test"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae"]==1)
    
])

tp_ae_test_2_weeks = len(df[
    (df["pos_test_2_weeks"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae"]==1)
    
])
print(f'False positives (FDR) - A&E attendance and recent positive test (2 weeks): {fp_ae_test_2_weeks} ({round((fp_ae_test_2_weeks/(fp_ae_test_2_weeks + tp_ae_test_2_weeks))*100,2)}%)')
print(f'False positives (FDR) - A&E attendance and recent positive test (4 weeks): {fp_ae_test} ({round((fp_ae_test/(fp_ae_test + tp_ae_test))*100,2)}%)')

positive_test_2_weeks = [len(ae_pos_test_before_2_weeks), round((len(ae_pos_test_before_2_weeks)/num_emergency_cov_admissions)*100, 2), fp_ae_test_2_weeks, tp_ae_test_2_weeks, round((fp_ae_test_2_weeks/(fp_ae_test_2_weeks + tp_ae_test_2_weeks))*100,2)]
positive_test_4_weeks = [len(ae_pos_test_before), round((len(ae_pos_test_before)/num_emergency_cov_admissions)*100, 2), fp_ae_test, tp_ae_test, round((fp_ae_test/(fp_ae_test + tp_ae_test))*100,2)]


####  Combining approaches - A&E attendance with hospital discharge 

##### Diagnosis codes (resp or cov) with recent positive test

In [ ]:
ae_hos_resp_cov_plus_test_2_weeks = (df.loc[(
        (df["ae_before_with_covid"]==1) | 
        (df["ae_before_with_resp"]==1) 
    )&
    (df["ae_before_with_positive_covid_test_2_weeks"]==1)
, "patient_id"])

fp_ae_hos_resp_cov_plus_test_2_weeks = len(df[
    (
        (df["ae_cov"]==1) |
        (df["ae_resp"]==1)    
    )&
    (df["pos_test_2_weeks"]==1)&
    (df["emergency_covid_hospital_admission_after_ae"]==0)
    
])

tp_ae_hos_resp_cov_plus_test_2_weeks = len(df[
    (
        (df["ae_cov"]==1) |
        (df["ae_resp"]==1)    
    )&
    (df["pos_test_2_weeks"]==1)&
    (df["emergency_covid_hospital_admission_after_ae"]==1)
    
])

print(f'Number with A&E attendance with either covid or respiratory diagnosis code and recent positive test (2 weeks) before admission: {len(ae_hos_resp_cov_plus_test_2_weeks)} ({round((len(ae_hos_resp_cov_plus_test_2_weeks)/num_emergency_cov_admissions)*100, 2)}%)')
print(f'False positives (FDR): {fp_ae_hos_resp_cov_plus_test_2_weeks} ({round((fp_ae_hos_resp_cov_plus_test_2_weeks/(fp_ae_hos_resp_cov_plus_test_2_weeks + tp_ae_hos_resp_cov_plus_test_2_weeks))*100,2)}%)')
code_plus_test = [len(ae_hos_resp_cov_plus_test_2_weeks), round((len(ae_hos_resp_cov_plus_test_2_weeks)/num_emergency_cov_admissions)*100, 2), fp_ae_hos_resp_cov_plus_test_2_weeks, tp_ae_hos_resp_cov_plus_test_2_weeks, round((fp_ae_hos_resp_cov_plus_test_2_weeks/(fp_ae_hos_resp_cov_plus_test_2_weeks + tp_ae_hos_resp_cov_plus_test_2_weeks))*100,2)]

##### Covid in primary care with recent positive test

In [ ]:
ae_hosp_prim_care_before_plus_2_week_test = (df.loc[(
        (df["ae_before_with_prim_care_suspected"]==1) | 
        (df["ae_before_with_prim_care_probable"]==1) 
    )
&
    (df["ae_before_with_positive_covid_test_2_weeks"]==1)
, "patient_id"])

fp_ae_hosp_prim_care_before_plus_2_week_test = len(df[
    (
        (df["covid_primary_care_before_ae"]==1) |
        (df["suspected_covid_primary_care_before_ae"]==1)    
    )&
    (df["pos_test_2_weeks"]==1)&
    (df["emergency_covid_hospital_admission_after_ae"]==0)
    
])

tp_ae_hosp_prim_care_before_plus_2_week_test = len(df[
    (
        (df["covid_primary_care_before_ae"]==1) |
        (df["suspected_covid_primary_care_before_ae"]==1)    
    )&
    (df["pos_test_2_weeks"]==1)&
    (df["emergency_covid_hospital_admission_after_ae"]==1)
    
])

print(f'Number with A&E attendance with suspected or probable covid in primary care and recent positive test (2 weeks) before admission: {len(ae_hosp_prim_care_before_plus_2_week_test)} ({round((len(ae_hosp_prim_care_before_plus_2_week_test)/num_emergency_cov_admissions)*100, 2)}%)')
print(f'False positives (FDR): {fp_ae_hosp_prim_care_before_plus_2_week_test} ({round((fp_ae_hosp_prim_care_before_plus_2_week_test/(fp_ae_hosp_prim_care_before_plus_2_week_test + tp_ae_hosp_prim_care_before_plus_2_week_test))*100,2)}%)')
primary_care_plus_test = [len(ae_hosp_prim_care_before_plus_2_week_test), round((len(ae_hosp_prim_care_before_plus_2_week_test)/num_emergency_cov_admissions)*100, 2), fp_ae_hosp_prim_care_before_plus_2_week_test, tp_ae_hosp_prim_care_before_plus_2_week_test, round((fp_ae_hosp_prim_care_before_plus_2_week_test/(fp_ae_hosp_prim_care_before_plus_2_week_test + tp_ae_hosp_prim_care_before_plus_2_week_test))*100,2)]

##### Recent positive test and either covid in primary care or diagnosis code (resp or cov)

In [ ]:
combined_before = (df.loc[
        (df["ae_before_with_positive_covid_test_2_weeks"]==1) &
        (
            (df["ae_before_with_prim_care_suspected"]==1) | 
            (df["ae_before_with_prim_care_probable"]==1) |
            (df["ae_before_with_covid"]==1) | 
            (df["ae_before_with_resp"]==1)   
        )
, "patient_id"])
fp_combined_before = len(df[
    (
        (df["covid_primary_care_before_ae"]==1) |
        (df["suspected_covid_primary_care_before_ae"]==1) |
        (df["ae_cov"]==1) |
        (df["ae_resp"]==1)
        
        
    )&
    (df["pos_test_2_weeks"]==1)&
    (df["emergency_covid_hospital_admission_after_ae"]==0)
    
])

tp_combined_before = len(df[
    (
        (df["covid_primary_care_before_ae"]==1) |
        (df["suspected_covid_primary_care_before_ae"]==1) |
        (df["ae_cov"]==1) |
        (df["ae_resp"]==1)
    )&
    (df["pos_test_2_weeks"]==1)&
    (df["emergency_covid_hospital_admission_after_ae"]==1)
    
])


print(f'Number with A&E attendance with recent positive test and either suspected or probable covid in primary care or a covid or respiratory diagnosis code on attendance: {len(combined_before)} ({round((len(combined_before)/num_emergency_cov_admissions)*100, 2)}%)')
print(f'False positives (FDR): {fp_combined_before} ({round((fp_combined_before/(fp_combined_before + tp_combined_before))*100,2)}%)')

pos_test_prim_care_or_diagnosis_code = [len(combined_before), round((len(combined_before)/num_emergency_cov_admissions)*100, 2), fp_combined_before, tp_combined_before, round((fp_combined_before/(fp_combined_before + tp_combined_before))*100,2)]

In [ ]:
identification_table_index = ["count", "sens", "fp", "tp", "fdr"]
identification_table = pd.DataFrame({
    "discharge_destination": pd.Series(discharge_destination,index=identification_table_index),
    "covid_diagnosis_code": pd.Series(diagnosis_code_cov,index=identification_table_index),
    "resp_diagnosis_code": pd.Series(diagnosis_code_resp,index=identification_table_index),
    "primary_care_suspected": pd.Series(prim_care_suspected,index=identification_table_index),
    "primary_care_probable": pd.Series(prim_care_probable,index=identification_table_index),
    "positive_test_2_weeks": pd.Series(positive_test_2_weeks,index=identification_table_index),
    "positive_test_4_weeks": pd.Series(positive_test_4_weeks,index=identification_table_index),
    "diagnosis_code_with_positive_test": pd.Series(code_plus_test,index=identification_table_index),
    "primary_care_with_positive_test": pd.Series(primary_care_plus_test,index=identification_table_index),
    "pos_test_prim_care_or_diagnosis": pd.Series(pos_test_prim_care_or_diagnosis_code,index=identification_table_index)
    
})
identification_table.to_csv("output/identification_table_no_hosp_discharge_primary.csv")

In [ ]:
produce_matrix(columns_list = [ae_hosp_before, ae_cov_before, ae_resp_before, ae_prim_care_probable_before, ae_prim_care_suspected_before, ae_pos_test_before_2_weeks, ae_pos_test_before, ae_hos_resp_cov_plus_test_2_weeks, ae_hosp_prim_care_before_plus_2_week_test], names=["Hospital discharge", "Covid diagnosis code", "Respiratory diagnosis code", "Probable covid code", "Suspected covid code", "Recent positive test (2 weeks)", "Recent positive test (4 weeks)", "Diagnosis codes (resp or cov) with recent positive test", "Covid in primary care with recent positive test", "Recent positive test and either covid in primary care or diagnosis code (resp or cov)"], filename="cm_no_hosp_discharge_primary")

### Restrict to A&E attendance with hospital discharge

In [ ]:
ae_with_hosp_discharge = len(df[
    (df["ae_attendance_hosp_discharge_date"].notnull())
])
print(f'Number of A&E attendances with hospital discharge: {ae_with_hosp_discharge}')

#### Method 1 - Discharge destination
Restrict to those that attended A&E and had a hospital discharge destination (hospital ward, emergency short stay ward, icu).

In [ ]:
ae_hosp_before = (df.loc[
    df["ae_before_with_hospital_discharge_date"].notnull()
, "patient_id"])

print(f'Number with A&E attendance with hospital discharge before admission: {len(ae_hosp_before)} ({round((len(ae_hosp_before)/num_emergency_cov_admissions)*100, 2)}%)')


# false posisitves
fp_ae_hosp_discharge = len(df[
    (df["ae_attendance_hosp_discharge_date"].notnull()) & 
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==0)

])

tp_ae_hosp_discharge = len(df[
    (df["ae_attendance_hosp_discharge_date"].notnull()) & 
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==1)

])
discharge_destination = [len(ae_hosp_before), round((len(ae_hosp_before)/num_emergency_cov_admissions)*100, 2),fp_ae_hosp_discharge, tp_ae_hosp_discharge, round((fp_ae_hosp_discharge/(fp_ae_hosp_discharge + tp_ae_hosp_discharge))*100,2)]

print(f'False positives (FDR): {fp_ae_hosp_discharge} ({round((fp_ae_hosp_discharge/(fp_ae_hosp_discharge + tp_ae_hosp_discharge))*100,2)}%)')

#### Method 2 - Diagnosis code
Restrict to those with a COVID-19 diagnosis code or respiratory-related diagnosis code from the [emergency care diagnosis refset](https://termbrowser.nhs.uk/?perspective=full&conceptId1=991411000000109&edition=uk-edition&release=v20220216&server=https://termbrowser.nhs.uk/sct-browser-api/snomed&langRefset=900000000000508004)

In [ ]:
ae_cov_before = (df.loc[
    df["ae_before_with_hospital_discharge_covid"]==1
, "patient_id"])

ae_resp_before = (df.loc[
    df["ae_before_with_hospital_discharge_resp"]==1
, "patient_id"])

print(f'Number with A&E attendance and covid diagnosis code before admission: {len(ae_cov_before)} ({round((len(ae_cov_before)/num_emergency_cov_admissions)*100, 2)}%)')
print(f'Number with A&E attendance and respiratory code before admission: {len(ae_resp_before)} ({round((len(ae_resp_before)/num_emergency_cov_admissions)*100, 2)}%)')

fp_ae_cov = len(df[
    (df["ae_cov_hosp_discharge"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==0)
    
])

fp_ae_resp = len(df[
    (df["ae_resp_hosp_discharge"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==0)
    
])

tp_ae_cov = len(df[
    (df["ae_cov_hosp_discharge"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==1)
    
])

tp_ae_resp = len(df[
    (df["ae_resp_hosp_discharge"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==1)
    
])

diagnosis_code_cov = [len(ae_cov_before), round((len(ae_cov_before)/num_emergency_cov_admissions)*100, 2), fp_ae_cov, tp_ae_cov, round((fp_ae_cov/(fp_ae_cov + tp_ae_cov))*100,2)]
diagnosis_code_resp = [len(ae_resp_before), round((len(ae_resp_before)/num_emergency_cov_admissions)*100, 2), fp_ae_resp, tp_ae_resp, round((len(ae_resp_before)/num_emergency_cov_admissions)*100, 2)]


print(f'False positives (FDR) - A&E attendance and covid diagnosis code: {fp_ae_cov} ({round((fp_ae_cov/(fp_ae_cov + tp_ae_cov))*100,2)}%)')
print(f'False positives (FDR) - A&E attendance and respiratory diagnosis code: {fp_ae_resp} ({round((fp_ae_resp/(fp_ae_resp + tp_ae_resp))*100,2)}%)')

#### Method 3 - Identification in primary care

Restrict to those with a probable or suspected COVID-19 code recorded in primary care in the 2 weeks prior to A&E attendance.

In [ ]:
ae_prim_care_probable_before = (df.loc[
    df["ae_before_with_hospital_discharge_prim_care_probable"]==1
, "patient_id"])

ae_prim_care_suspected_before = (df.loc[
    df["ae_before_with_hospital_discharge_prim_care_suspected"]==1
, "patient_id"])

print(f'Number with A&E attendance and recent suspected covid code in primary care before admission: {len(ae_prim_care_suspected_before)} ({round((len(ae_prim_care_suspected_before)/num_emergency_cov_admissions)*100, 2)}%)')
print(f'Number with A&E attendance and recent probable covid code in primary care before admission: {len(ae_prim_care_probable_before)} ({round((len(ae_prim_care_probable_before)/num_emergency_cov_admissions)*100, 2)}%)')

fp_ae_suspected = len(df[
    (df["suspected_covid_primary_care_before_ae_hosp_discharge"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==0)    
])

fp_ae_probable = len(df[
    (df["covid_primary_care_before_ae_hosp_discharge"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==0)
])

tp_ae_suspected = len(df[
    (df["suspected_covid_primary_care_before_ae_hosp_discharge"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==1)    
])

tp_ae_probable = len(df[
    (df["covid_primary_care_before_ae_hosp_discharge"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==1)
])


prim_care_suspected = [len(ae_prim_care_suspected_before), round((len(ae_prim_care_suspected_before)/num_emergency_cov_admissions)*100, 2), fp_ae_suspected, tp_ae_suspected, round((fp_ae_suspected/(fp_ae_suspected + tp_ae_suspected))*100,2)]
prim_care_probable = [len(ae_prim_care_probable_before), round((len(ae_prim_care_probable_before)/num_emergency_cov_admissions)*100, 2), fp_ae_probable, tp_ae_probable, round((fp_ae_probable/(fp_ae_probable + tp_ae_probable))*100,2)]

print(f'False positives (FDR) - A&E attendance and probable covid code in primary care: {fp_ae_probable} ({round((fp_ae_probable/(fp_ae_probable + tp_ae_probable))*100,2)}%)')
print(f'False positives (FDR) - A&E attendance and suspected covid code in primary care: {fp_ae_suspected} ({round((fp_ae_suspected/(fp_ae_suspected + tp_ae_suspected))*100,2)}%)')

#### Method 4 - Positive test result

Restrict to those with a positive COVID-19 test result recorded up to 4 weeks prior to A&E attendance.

In [ ]:
ae_pos_test_before_2_weeks = (df.loc[
    df["ae_before_with_hospital_discharge_positive_covid_test_2_weeks"]==1
, "patient_id"])

ae_pos_test_before = (df.loc[
    df["ae_before_with_hospital_discharge_positive_covid_test"]==1
, "patient_id"])


print(f'Number with A&E attendance and recent positive test (2 weeks) before admission: {len(ae_pos_test_before_2_weeks)} ({round((len(ae_pos_test_before_2_weeks)/num_emergency_cov_admissions)*100, 2)}%)')
print(f'Number with A&E attendance and recent positive test (4 weeks) before admissio: {len(ae_pos_test_before)} ({round((len(ae_pos_test_before)/num_emergency_cov_admissions)*100, 2)}%)')

fp_ae_test = len(df[
    (df["pos_test_hosp_discharge"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==0)
    
])

fp_ae_test_2_weeks = len(df[
    (df["pos_test_2_weeks_hosp_discharge"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==0)
    
])

tp_ae_test = len(df[
    (df["pos_test_hosp_discharge"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==1)
    
])

tp_ae_test_2_weeks = len(df[
    (df["pos_test_2_weeks_hosp_discharge"]==1) & 
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==1)
    
])

positive_test_2_weeks = [len(ae_pos_test_before_2_weeks), round((len(ae_pos_test_before_2_weeks)/num_emergency_cov_admissions)*100, 2), fp_ae_test_2_weeks, tp_ae_test_2_weeks, round((fp_ae_test_2_weeks/(fp_ae_test_2_weeks + tp_ae_test_2_weeks))*100,2)]
positive_test_4_weeks = [len(ae_pos_test_before), round((len(ae_pos_test_before)/num_emergency_cov_admissions)*100, 2), fp_ae_test, tp_ae_test, round((fp_ae_test/(fp_ae_test + tp_ae_test))*100,2)]


print(f'False positives (FDR) - A&E attendance and recent positive test (2 weeks): {fp_ae_test_2_weeks} ({round((fp_ae_test_2_weeks/(fp_ae_test_2_weeks + tp_ae_test_2_weeks))*100,2)}%)')
print(f'False positives (FDR) - A&E attendance and recent positive test (4 weeks): {fp_ae_test} ({round((fp_ae_test/(fp_ae_test + tp_ae_test))*100,2)}%)')

####  Combining approaches - A&E attendance with hospital discharge 

##### Diagnosis codes (resp or cov) with recent positive test

In [ ]:
ae_hos_resp_cov_plus_test_2_weeks = (df.loc[(
        (df["ae_before_with_hospital_discharge_covid"]==1) | 
        (df["ae_before_with_hospital_discharge_resp"]==1) 
    )&
    (df["ae_before_with_hospital_discharge_positive_covid_test_2_weeks"]==1)
, "patient_id"])

fp_ae_hos_resp_cov_plus_test_2_weeks = len(df[
    (
        (df["ae_cov_hosp_discharge"]==1) |
        (df["ae_resp_hosp_discharge"]==1)    
    )&
    (df["pos_test_2_weeks_hosp_discharge"]==1)&
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==0)
    
])

tp_ae_hos_resp_cov_plus_test_2_weeks = len(df[
    (
        (df["ae_cov_hosp_discharge"]==1) |
        (df["ae_resp_hosp_discharge"]==1)    
    )&
    (df["pos_test_2_weeks_hosp_discharge"]==1)&
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==1)
    
])

print(f'Number with A&E attendance with either covid or respiratory diagnosis code and recent positive test (2 weeks) before admission: {len(ae_hos_resp_cov_plus_test_2_weeks)} ({round((len(ae_hos_resp_cov_plus_test_2_weeks)/num_emergency_cov_admissions)*100, 2)}%)')
print(f'False positives (FDR): {fp_ae_hos_resp_cov_plus_test_2_weeks} ({round((fp_ae_hos_resp_cov_plus_test_2_weeks/(fp_ae_hos_resp_cov_plus_test_2_weeks + tp_ae_hos_resp_cov_plus_test_2_weeks))*100,2)}%)')

code_plus_test = [len(ae_hos_resp_cov_plus_test_2_weeks), round((len(ae_hos_resp_cov_plus_test_2_weeks)/num_emergency_cov_admissions)*100, 2), fp_ae_hos_resp_cov_plus_test_2_weeks, tp_ae_hos_resp_cov_plus_test_2_weeks, round((fp_ae_hos_resp_cov_plus_test_2_weeks/(fp_ae_hos_resp_cov_plus_test_2_weeks + tp_ae_hos_resp_cov_plus_test_2_weeks))*100,2)]

##### Covid in primary care with recent positive test

In [ ]:
ae_hosp_prim_care_before_plus_2_week_test = (df.loc[(
        (df["ae_before_with_hospital_discharge_prim_care_suspected"]==1) | 
        (df["ae_before_with_hospital_discharge_prim_care_probable"]==1) 
    )
&
    (df["ae_before_with_hospital_discharge_positive_covid_test_2_weeks"]==1)
, "patient_id"])

fp_ae_hosp_prim_care_before_plus_2_week_test = len(df[
    (
        (df["covid_primary_care_before_ae_hosp_discharge"]==1) |
        (df["suspected_covid_primary_care_before_ae_hosp_discharge"]==1)    
    )&
    (df["pos_test_2_weeks_hosp_discharge"]==1)&
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==0)
    
])

tp_ae_hosp_prim_care_before_plus_2_week_test = len(df[
    (
        (df["covid_primary_care_before_ae_hosp_discharge"]==1) |
        (df["suspected_covid_primary_care_before_ae_hosp_discharge"]==1)    
    )&
    (df["pos_test_2_weeks_hosp_discharge"]==1)&
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==1)
    
])

print(f'Number with A&E attendance with suspected or probable covid in primary care and recent positive test (2 weeks) before admission: {len(ae_hosp_prim_care_before_plus_2_week_test)} ({round((len(ae_hosp_prim_care_before_plus_2_week_test)/num_emergency_cov_admissions)*100, 2)}%)')
print(f'False positives (FDR): {fp_ae_hosp_prim_care_before_plus_2_week_test} ({round((fp_ae_hosp_prim_care_before_plus_2_week_test/(fp_ae_hosp_prim_care_before_plus_2_week_test + tp_ae_hosp_prim_care_before_plus_2_week_test))*100,2)}%)')

primary_care_plus_test = [len(ae_hosp_prim_care_before_plus_2_week_test), round((len(ae_hosp_prim_care_before_plus_2_week_test)/num_emergency_cov_admissions)*100, 2), fp_ae_hosp_prim_care_before_plus_2_week_test, tp_ae_hosp_prim_care_before_plus_2_week_test, round((fp_ae_hosp_prim_care_before_plus_2_week_test/(fp_ae_hosp_prim_care_before_plus_2_week_test + tp_ae_hosp_prim_care_before_plus_2_week_test))*100,2)]

##### Recent positive test and either covid in primary care or diagnosis code (resp or cov)

In [ ]:
combined_before = (df.loc[
        (df["ae_before_with_hospital_discharge_positive_covid_test_2_weeks"]==1) &
        (
            (df["ae_before_with_hospital_discharge_prim_care_suspected"]==1) | 
            (df["ae_before_with_hospital_discharge_prim_care_probable"]==1) |
            (df["ae_before_with_hospital_discharge_covid"]==1) | 
            (df["ae_before_with_hospital_discharge_resp"]==1)   
        )
, "patient_id"])
fp_combined_before = len(df[
    (
        (df["covid_primary_care_before_ae_hosp_discharge"]==1) |
        (df["suspected_covid_primary_care_before_ae_hosp_discharge"]==1) |
        (df["ae_cov_hosp_discharge"]==1) |
        (df["ae_resp_hosp_discharge"]==1)
        
        
    )&
    (df["pos_test_2_weeks_hosp_discharge"]==1)&
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==0)
    
])

tp_combined_before = len(df[
    (
        (df["covid_primary_care_before_ae_hosp_discharge"]==1) |
        (df["suspected_covid_primary_care_before_ae_hosp_discharge"]==1) |
        (df["ae_cov_hosp_discharge"]==1) |
        (df["ae_resp_hosp_discharge"]==1)
    )&
    (df["pos_test_2_weeks_hosp_discharge"]==1)&
    (df["emergency_covid_hospital_admission_after_ae_hosp_discharge"]==1)
    
])


print(f'Number with A&E attendance with recent positive test and either suspected or probable covid in primary care or a covid or respiratory diagnosis code on attendance: {len(combined_before)} ({round((len(combined_before)/num_emergency_cov_admissions)*100, 2)}%)')
print(f'False positives (FDR): {fp_combined_before} ({round((fp_combined_before/(fp_combined_before + tp_combined_before))*100,2)}%)')

pos_test_prim_care_or_diagnosis_code = [len(combined_before), round((len(combined_before)/num_emergency_cov_admissions)*100, 2), fp_combined_before, tp_combined_before, round((fp_combined_before/(fp_combined_before + tp_combined_before))*100,2)]

In [ ]:
identification_table_index = ["count", "sens", "fp", "tp", "fdr"]
identification_table = pd.DataFrame({
    "discharge_destination": pd.Series(discharge_destination,index=identification_table_index),
    "covid_diagnosis_code": pd.Series(diagnosis_code_cov,index=identification_table_index),
    "resp_diagnosis_code": pd.Series(diagnosis_code_resp,index=identification_table_index),
    "primary_care_suspected": pd.Series(prim_care_suspected,index=identification_table_index),
    "primary_care_probable": pd.Series(prim_care_probable,index=identification_table_index),
    "positive_test_2_weeks": pd.Series(positive_test_2_weeks,index=identification_table_index),
    "positive_test_4_weeks": pd.Series(positive_test_4_weeks,index=identification_table_index),
    "diagnosis_code_with_positive_test": pd.Series(code_plus_test,index=identification_table_index),
    "primary_care_with_positive_test": pd.Series(primary_care_plus_test,index=identification_table_index),
    "pos_test_prim_care_or_diagnosis": pd.Series(pos_test_prim_care_or_diagnosis_code,index=identification_table_index)
    
})
identification_table.to_csv("output/identification_table_hosp_discharge_primary.csv")

In [ ]:
produce_matrix(columns_list = [ae_hosp_before, ae_cov_before, ae_resp_before, ae_prim_care_probable_before, ae_prim_care_suspected_before, ae_pos_test_before_2_weeks, ae_pos_test_before, ae_hos_resp_cov_plus_test_2_weeks, ae_hosp_prim_care_before_plus_2_week_test], names=["Hospital discharge", "Covid diagnosis code", "Respiratory diagnosis code", "Probable covid code", "Suspected covid code", "Recent positive test (2 weeks)", "Recent positive test (4 weeks)", "Diagnosis codes (resp or cov) with recent positive test", "Covid in primary care with recent positive test", "Recent positive test and either covid in primary care or diagnosis code (resp or cov)"], filename="cm_hosp_discharge_primary")